# IAA primera etapa

En esta notebook vamos a ver cuánto estuvieron de acuerdo los etiquetadores 

Primero, carguemos el excel con las etiquetas

In [29]:
import pandas as pd
import glob
import numpy as np
import re


paths = glob.glob("../data/etiquetados_final/*.xlsx")

dfs = []
annotators = []

for path in paths:
    name = path[path.find("(")+1:path.find(")")]
    
    df = pd.read_excel(path)
    df = df.iloc[3:]
    annotators.append(name)

    df.rename(columns={df.columns[0]: "id", "TITULO": "titulo", " ": "etiquetado", "Etiquetado": "etiquetado"}, inplace=True)

    df["anotador"] = name
    dfs.append(df)
    
annotators = sorted(annotators)
annotator_df = pd.concat(dfs).set_index(["anotador", "id"])

In [31]:
label_cols = ['TARGET A', 'TARGET B', 'TARGET C', 'TARGET D',
       'TARGET E', 'TARGET F']


In [33]:
idx = pd.MultiIndex.from_product([label_cols, annotators])
df = pd.DataFrame(index=idx)

id2title = dfs[0][["id", "titulo"]].set_index("id").to_dict()["titulo"]


for idx, row in annotator_df.iterrows():
    for col in label_cols:
        if type(row[col]) is str:
            row[col] = row[col].strip().upper()
            
            if row[col] not in {"POS", "NEG", "NEU"}:
                row[col] = float('nan')
                
            df.loc[(col, idx[0]), idx[1]] = row[col]
df

23    24    31    39    75     \
TARGET A ../data/etiquetados_final/e2.xls    NaN   NaN   NaN   NaN   NaN   
         ../data/etiquetados_final/e3.xls    NaN   NaN   NaN   NaN   NaN   
         ../data/etiquetados_final/et1.xls   NaN   NaN   NaN   NaN   NaN   
TARGET B ../data/etiquetados_final/e2.xls    NaN   NaN   NaN   NaN   NEG   
         ../data/etiquetados_final/e3.xls    NaN   NaN   NaN   NaN   NEG   
         ../data/etiquetados_final/et1.xls   NaN   NaN   NaN   NaN   NEG   
TARGET C ../data/etiquetados_final/e2.xls    NaN   NaN   NaN   POS   NaN   
         ../data/etiquetados_final/e3.xls    NaN   NaN   NaN   POS   NaN   
         ../data/etiquetados_final/et1.xls   NaN   NaN   NaN   POS   NaN   
TARGET D ../data/etiquetados_final/e2.xls    POS   POS   NEU   NaN   NaN   
         ../data/etiquetados_final/e3.xls    POS   POS   POS   NaN   NaN   
         ../data/etiquetados_final/et1.xls   NEG   POS   NEU   NaN   NaN   
TARGET E ../data/etiquetados_final/e2.xls    NaN   NaN   NaN   NEG   NaN   
         ../data/etiquetados_final/e3.xls    NaN   NaN   NaN   NEG   NaN   
         ../data/etiquetados_final/et1.xls   NaN   NaN   NaN   NEG   NaN   
TARGET F ../data/etiquetados_final/e2.xls    NaN   NaN   NaN   NaN   NaN   
         ../data/etiquetados_final/e3.xls    NaN   NaN   NaN   NaN   NaN   
         ../data/etiquetados_final/et1.xls   NaN   NaN   NaN   NaN   NaN   

                                           77    78    82    84    90     ...  \
TARGET A ../data/etiquetados_final/e2.xls    NaN   NaN   NaN   NaN   NaN  ...   
         ../data/etiquetados_final/e3.xls    NaN   NaN   NaN   NaN   NaN  ...   
         ../data/etiquetados_final/et1.xls   NaN   NaN   NaN   NaN   NaN  ...   
TARGET B ../data/etiquetados_final/e2.xls    NaN   NaN   NaN   NEG   NaN  ...   
         ../data/etiquetados_final/e3.xls    NaN   NaN   NaN   POS   NaN  ...   
         ../data/etiquetados_final/et1.xls   NaN   NaN   NaN   NEG   NaN  ...   
TARGET C ../data/etiquetados_final/e2.xls    NaN   NaN   NaN   NaN   NaN  ...   
         ../data/etiquetados_final/e3.xls    NaN   NaN   NaN   NaN   NaN  ...   
         ../data/etiquetados_final/et1.xls   NaN   NaN   NaN   NaN   NaN  ...   
TARGET D ../data/etiquetados_final/e2.xls    NaN   POS   POS   NaN   NaN  ...   
         ../data/etiquetados_final/e3.xls    NaN   POS   POS   NaN   NaN  ...   
         ../data/etiquetados_final/et1.xls   NaN   POS   POS   NaN   NaN  ...   
TARGET E ../data/etiquetados_final/e2.xls    POS   NaN   NaN   NaN   NEG  ...   
         ../data/etiquetados_final/e3.xls    POS   NaN   NaN   NaN   NEG  ...   
         ../data/etiquetados_final/et1.xls   NEU   NaN   NaN   NaN   NEG  ...   
TARGET F ../data/etiquetados_final/e2.xls    NaN   NaN   NaN   NaN   NaN  ...   
         ../data/etiquetados_final/e3.xls    NaN   NaN   NaN   NaN   NaN  ...   
         ../data/etiquetados_final/et1.xls   NaN   NaN   NaN   NaN   NaN  ...   

                                           11982 12056 12082 12749 13076  \
TARGET A ../data/etiquetados_final/e2.xls    NaN   NaN   NaN   NaN   NaN   
         ../data/etiquetados_final/e3.xls    NaN   NaN   NaN   NaN   NaN   
         ../data/etiquetados_final/et1.xls   NaN   NaN   NaN   NaN   NaN   
TARGET B ../data/etiquetados_final/e2.xls    NaN   NaN   NaN   NaN   NaN   
         ../data/etiquetados_final/e3.xls    NaN   NaN   NaN   NaN   NaN   
         ../data/etiquetados_final/et1.xls   NaN   NaN   NaN   NaN   NaN   
TARGET C ../data/etiquetados_final/e2.xls    NaN   NaN   NaN   NaN   NaN   
         ../data/etiquetados_final/e3.xls    NaN   NaN   NaN   NaN   NaN   
         ../data/etiquetados_final/et1.xls   NaN   NaN   NaN   NaN   NaN   
TARGET D ../data/etiquetados_final/e2.xls    NaN   NaN   NaN   NaN   NaN   
         ../data/etiquetados_final/e3.xls    NaN   NaN   NaN   NaN   NaN   
         ../data/etiquetados_final/et1.xls   NaN   NaN   NaN   NaN   NaN   
TARGET E ../data/etiquetados_final/e2.xls    NaN   NaN   NaN   NaN   N

In [34]:
import krippendorff

for label in label_cols:
    print(label)
    mat = df.loc[label].notna().values
    mat = mat.astype(float)
    alpha = krippendorff.alpha(mat)
    
    support = (mat.sum(axis=0) > 0).sum()
    print(f"Alpha sobre etiquetar {label:<12}: {alpha:.3f} (soporte {support})")


TARGET A
Alpha sobre etiquetar TARGET A    : 0.989 (soporte 30)
TARGET B
Alpha sobre etiquetar TARGET B    : 0.994 (soporte 124)
TARGET C
Alpha sobre etiquetar TARGET C    : 0.990 (soporte 1070)
TARGET D
Alpha sobre etiquetar TARGET D    : 0.988 (soporte 216)
TARGET E
Alpha sobre etiquetar TARGET E    : 0.980 (soporte 984)
TARGET F
Alpha sobre etiquetar TARGET F    : 1.000 (soporte 15)


# Agreements

Veamos qué tanto acuerdo hay por cada fuerza. Probemos primero el agreement ordinal (es decir, que es peor decir NEG y POS que NEG y NEU, por ejemplo)

In [35]:
from sklearn.metrics import cohen_kappa_score

for label in label_cols:
    print(label)
    mat = df.loc[label].values
    mat[mat == 'NEG'] = -1
    mat[mat == 'POS'] = 1
    mat[mat == 'NEU'] = 0
    mat = mat.astype(float)
    alpha = krippendorff.alpha(mat, level_of_measurement="ordinal")
    
    support = ((~np.isnan(mat)).sum(axis=0) > 0).sum()
    print(f"Alpha sobre etiquetar {label:<12}: {alpha:.3f} (soporte {support})")


TARGET A
Alpha sobre etiquetar TARGET A    : 0.456 (soporte 30)
TARGET B
Alpha sobre etiquetar TARGET B    : 0.441 (soporte 124)
TARGET C
Alpha sobre etiquetar TARGET C    : 0.619 (soporte 1070)
TARGET D
Alpha sobre etiquetar TARGET D    : 0.464 (soporte 216)
TARGET E
Alpha sobre etiquetar TARGET E    : 0.598 (soporte 984)
TARGET F
Alpha sobre etiquetar TARGET F    : 0.090 (soporte 15)


Antes nos dio esto:

- FdT
Alpha sobre etiquetar FdT         : 0.738 (soporte 73)
- JxC
Alpha sobre etiquetar JxC         : 0.780 (soporte 72)
- Lavagna
Alpha sobre etiquetar Lavagna     : 0.404 (soporte 7)
- FIT
Alpha sobre etiquetar FIT         : 0.749 (soporte 15)
- Espert
Alpha sobre etiquetar Espert      : 0.417 (soporte 3)
- Centurión
Alpha sobre etiquetar Centurión   : nan (soporte 1)

Con match exacto

In [36]:
from sklearn.metrics import cohen_kappa_score

for label in label_cols:
    print(label)
    mat = df.loc[label].values
    mat[mat == 'NEG'] = -1
    mat[mat == 'POS'] = 1
    mat[mat == 'NEU'] = 0
    mat = mat.astype(float)
    alpha = krippendorff.alpha(mat, level_of_measurement="nominal")
    
    support = ((~np.isnan(mat)).sum(axis=0) > 0).sum()
    print(f"Alpha sobre etiquetar {label:<12}: {alpha:.3f} (soporte {support})")


TARGET A
Alpha sobre etiquetar TARGET A    : 0.344 (soporte 30)
TARGET B
Alpha sobre etiquetar TARGET B    : 0.389 (soporte 124)
TARGET C
Alpha sobre etiquetar TARGET C    : 0.507 (soporte 1070)
TARGET D
Alpha sobre etiquetar TARGET D    : 0.419 (soporte 216)
TARGET E
Alpha sobre etiquetar TARGET E    : 0.478 (soporte 984)
TARGET F
Alpha sobre etiquetar TARGET F    : 0.155 (soporte 15)


Antes nos dio...

- FdT
Alpha sobre etiquetar FdT         : 0.668 (soporte 73)
- JxC
Alpha sobre etiquetar JxC         : 0.667 (soporte 72)
- Lavagna
Alpha sobre etiquetar Lavagna     : 0.403 (soporte 7)
- FIT
Alpha sobre etiquetar FIT         : 0.625 (soporte 15)
- Espert
Alpha sobre etiquetar Espert      : 0.200 (soporte 3)
- Centurión
Alpha sobre etiquetar Centurión   : nan (soporte 1)
